In [92]:
import lightkurve as lk
import pandas as pd
import numpy as np
import warnings

In [39]:
tic = 4289780

search = lk.search_lightcurve(target=f'TIC {tic}', author='SPOC', exptime=120)
search

#,mission,year,author,exptime,target_name,distance
,,,,s,,arcsec
0,TESS Sector 06,2018,SPOC,120,4289780,0.0
1,TESS Sector 32,2020,SPOC,120,4289780,0.0


In [5]:
file = pd.read_csv('/../NAOS/Bstars_v3/TIC_377_test.csv', sep=';')
id_list = file.TIC

sector_dict = {}

for i in range(len(id_list)):
    search = lk.search_lightcurve(target=f'TIC {id_list[i]}', author='SPOC', exptime=120)
    sector_list = []
    for j in range(len(search)):
        sector = search.table['mission'][j][12:]
        sector_list.append(sector)
        
    sector_dict[f'{id_list[i]}'] = sector_list
    
sector_dict

{'1152358': ['22'],
 '4289780': ['06', '32'],
 '4373827': ['20', '44', '45', '46', '47', '60']}

In [93]:
file = pd.read_csv('/../NAOS/Bstars_v3/TIC_377_test.csv', sep=';')
id_list = file.TIC

tic_list = []
sector_list = []

for i in range(len(id_list)):
    search = lk.search_lightcurve(target=f'TIC {id_list[i]}', author='SPOC', exptime=120)
    for j in range(len(search)):
        sector = search.table['mission'][j][12:]
        target = search.table['target_name'][j]
        tic_list.append(f'{target}'.zfill(10))
        sector_list.append(sector)
        
df_target = pd.DataFrame(tic_list, columns=['TIC'])       
df_sector = pd.DataFrame(sector_list, columns=['Sectors'])
main_list = pd.concat([df_target, df_sector], axis=1)
main_list

,TIC,Sectors
0,0001152358,22
1,0004289780,06
2,0004289780,32
3,0004373827,20
4,0004373827,44
5,0004373827,45
6,0004373827,46
7,0004373827,47
8,0004373827,60


In [94]:
main_df = main_list.groupby('TIC').agg(lambda x: list(set(x))).reset_index()
main_df

,TIC,Sectors
0,0001152358,[22]
1,0004289780,"[06, 32]"
2,0004373827,"[45, 46, 20, 47, 44, 60]"


In [95]:
N_sector = main_list.groupby('TIC').count().reset_index()
N_sector

,TIC,Sectors
0,0001152358,1
1,0004289780,2
2,0004373827,6


In [96]:
final_df = pd.concat([main_df, N_sector.Sectors], axis=1)
final_df = final_df.set_axis(['TIC', 'Sectors', 'N_sectors'], axis=1)
for i in range(len(final_df)):
    final_df.Sectors[i] = sorted(final_df.Sectors[i])
    warnings.filterwarnings("ignore")
final_df

C:\Users\jcfis\AppData\Local\Temp\ipykernel_8512\1487661036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.Sectors[i] = sorted(final_df.Sectors[i])


,TIC,Sectors,N_sectors
0,0001152358,[22],1
1,0004289780,"[06, 32]",2
2,0004373827,"[20, 44, 45, 46, 47, 60]",6


In [84]:
sorted(final_df.Sectors[2])

['20', '44', '45', '46', '47', '60']

In [87]:
final_df.Sectors[2] = sorted(final_df.Sectors[2])
final_df

C:\Users\jcfis\AppData\Local\Temp\ipykernel_8512\2474643978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.Sectors[2] = sorted(final_df.Sectors[2])


,TIC,Sectors,N_sectors
0,0001152358,[22],1
1,0004289780,"[06, 32]",2
2,0004373827,"[20, 44, 45, 46, 47, 60]",6


In [27]:
df = pd.DataFrame.from_dict(sector_dict, orient='index')
df['TIC'] = df.index
df

,0,1,2,3,4,5,TIC
1152358,22,None,None,None,None,None,1152358
4289780,06,32,None,None,None,None,4289780
4373827,20,44,45,46,47,60,4373827
